# speech data에 넣기 위해 40초 단위로 wav 파일 자르기
# PCM 16-bit, 16kHz, mono 채널 형식으로 변환 

pip install ffmpeg
(mac homebrew로 ffmpeg 설치 : brew install ffmpeg)

ffmpeg web에서 Windows builds from gyan.dev 클릭 > full 다운로드 후 C:에 ffmpeg 폴더 내에 압축풀기>
PowerShell 에서 환경변수 추가 (관리자) > [System.Environment]::SetEnvironmentVariable("Path", $env:Path + ";C:\ffmpeg\bin", [System.EnvironmentVariableTarget]::Machine)
실행후 새로운 파워셸 창에서 
ffmpeg -version 로 확인
이후 vs code 가상환경 활성화 후 필요한거 설치 : 

In [6]:
import subprocess
import os

def convert_audio_for_azure(file_path, output_dir, target_sample_rate=16000):
    """
    FFmpeg를 사용하여 오디오 파일을 Azure Speech STT 학습용 포맷(PCM 16-bit, 16kHz, Mono)으로 변환
    - 파일명을 변경하지 않고 원본 이름 그대로 유지
    """
    os.makedirs(output_dir, exist_ok=True)
    file_name = os.path.basename(file_path)  # 원본 파일명 유지
    output_file = os.path.join(output_dir, file_name)  # 변환된 파일도 같은 이름 사용

    cmd = [
        "ffmpeg",
        "-i", file_path,                # 입력 파일
        "-ac", "1",                      # Mono 채널 변환
        "-ar", str(target_sample_rate),  # 샘플링 레이트 변환 (16kHz)
        "-acodec", "pcm_s16le",          # PCM 16-bit 변환
        output_file
    ]

    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"✅ 변환 완료: {output_file}")

    return output_file

def get_audio_duration(file_path):
    """
    FFmpeg를 사용하여 오디오 길이(초)를 가져옴
    """
    cmd = ["ffmpeg", "-i", file_path, "-f", "null", "-"]
    result = subprocess.run(cmd, stderr=subprocess.PIPE, text=True, encoding="utf-8")
    for line in result.stderr.split("\n"):
        if "Duration" in line:
            time_str = line.split("Duration: ")[1].split(",")[0]
            h, m, s = map(float, time_str.replace(":", " ").split())
            return h * 3600 + m * 60 + s
    return None

def split_audio_ffmpeg(file_path, output_dir, chunk_length_sec=40):
    """
    FFmpeg를 사용하여 40초 단위로 오디오를 자르고 저장
    - 원본 파일명을 유지한 채, "-01", "-02" 등으로 번호만 추가
    """
    file_name = os.path.basename(file_path).replace(".wav", "")  # 원본 파일명 유지
    os.makedirs(output_dir, exist_ok=True)

    duration = get_audio_duration(file_path)
    if duration is None:
        print("❌ 오디오 길이를 가져올 수 없습니다. 파일이 손상되었거나 FFmpeg 실행 오류일 수 있습니다.")
        return

    part = 1
    start = 0

    while start < duration:
        end = min(start + chunk_length_sec, duration)
        output_file = os.path.join(output_dir, f"{file_name}-{part:02d}.wav")  # 번호만 추가

        cmd = [
            "ffmpeg",
            "-i", file_path,
            "-ss", str(start),
            "-t", str(chunk_length_sec),
            "-c", "copy",
            output_file
        ]

        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"✅ {output_file} 저장 완료 ({start:.1f}s ~ {end:.1f}s)")

        start = end
        part += 1

    print(f"🔹 오디오가 성공적으로 {chunk_length_sec}초 단위로 잘라졌습니다! 저장 경로: {output_dir}")

def process_audio_for_azure(file_path):
    """
    1. Azure STT용 PCM 16-bit, 16kHz, Mono 변환 (파일명 유지)
    2. 40초 단위로 오디오 파일 분할 (파일명 유지)
    """
    base_dir = os.path.dirname(file_path)
    converted_dir = os.path.join(base_dir, "converted")
    split_dir = os.path.join(base_dir, "chunks")

    # 1. WAV 파일을 Azure STT 훈련용 포맷으로 변환 (파일명 유지)
    converted_audio = convert_audio_for_azure(file_path, converted_dir)

    # 2. 변환된 WAV 파일을 40초 단위로 분할 (파일명 유지)
    split_audio_ffmpeg(converted_audio, split_dir)

# 실행 예시
file_path = r".\Data\test_audio\test03\test_audio_03-07-현수민.wav"
process_audio_for_azure(file_path)


✅ 변환 완료: .\Data\test_audio\test03\converted\test_audio_03-07-현수민.wav
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-01.wav 저장 완료 (0.0s ~ 40.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-02.wav 저장 완료 (40.0s ~ 80.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-03.wav 저장 완료 (80.0s ~ 120.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-04.wav 저장 완료 (120.0s ~ 160.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-05.wav 저장 완료 (160.0s ~ 200.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-06.wav 저장 완료 (200.0s ~ 240.0s)
✅ .\Data\test_audio\test03\chunks\test_audio_03-07-현수민-07.wav 저장 완료 (240.0s ~ 243.9s)
🔹 오디오가 성공적으로 40초 단위로 잘라졌습니다! 저장 경로: .\Data\test_audio\test03\chunks


# 형식 변경 없이 자르기만 하는 버전

In [5]:
import subprocess
import os

def get_audio_duration(file_path):
    """
    FFmpeg를 사용하여 오디오 길이(초)를 가져옴
    """
    cmd = ["ffmpeg", "-i", file_path, "-f", "null", "-"]
    result = subprocess.run(cmd, stderr=subprocess.PIPE, text=True)
    for line in result.stderr.split("\n"):
        if "Duration" in line:
            time_str = line.split("Duration: ")[1].split(",")[0]
            h, m, s = map(float, time_str.replace(":", " ").split())
            return h * 3600 + m * 60 + s
    return None

def split_audio_ffmpeg(file_path, chunk_length_sec=40):
    """
    FFmpeg를 사용하여 40초 단위로 오디오를 자르고 저장
    """
    file_name = os.path.basename(file_path).replace(".wav", "")
    file_dir = os.path.dirname(file_path)

    output_dir = os.path.join(file_dir, "chunks")
    os.makedirs(output_dir, exist_ok=True)

    duration = get_audio_duration(file_path)
    if duration is None:
        print("❌ 오디오 길이를 가져올 수 없습니다. 파일이 손상되었거나 FFmpeg 실행 오류일 수 있습니다.")
        return

    part = 1
    start = 0

    while start < duration:
        end = min(start + chunk_length_sec, duration)
        output_file = os.path.join(output_dir, f"{file_name}-{part:02d}.wav")

        cmd = [
            "ffmpeg",
            "-i", file_path,
            "-ss", str(start),
            "-t", str(chunk_length_sec),
            "-c", "copy",
            output_file
        ]

        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"✅ {output_file} 저장 완료 ({start:.1f}s ~ {end:.1f}s)")

        start = end
        part += 1

    print(f"🔹 오디오가 성공적으로 {chunk_length_sec}초 단위로 잘라졌습니다! 저장 경로: {output_dir}")

# 실행 예시
file_path = r".\Data\test_audio\test_audio_02.wav"
split_audio_ffmpeg(file_path)


✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-01.wav 저장 완료 (0.0s ~ 40.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-02.wav 저장 완료 (40.0s ~ 80.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-03.wav 저장 완료 (80.0s ~ 120.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-04.wav 저장 완료 (120.0s ~ 160.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-05.wav 저장 완료 (160.0s ~ 200.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-06.wav 저장 완료 (200.0s ~ 240.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-07.wav 저장 완료 (240.0s ~ 280.0s)
✅ C:\AI-team3\Read-API\Data\test_audio\chunks\test_audio_02-08.wav 저장 완료 (280.0s ~ 297.6s)
🔹 오디오가 성공적으로 40초 단위로 잘라졌습니다! 저장 경로: C:\AI-team3\Read-API\Data\test_audio\chunks


# pause에서 끊는걸 시도해봤으나 결과적으로 실패. 아마 baseline 잡음 때문인 것 같기도 하고...

pip install ffmpeg-python numpy

####이 방법은 python 최신 버전에서 audioop이 더이상 내장되지 않기 때문에 못쓰는 것으로 판명됨####
pip install --upgrade pydub ffmpeg-python pyaudio audioread

업데이트된 pydub을 설치해야 audioop 없이 동작 가능해.
먼저, 기존 pydub을 삭제하고 새 버전을 설치하자.

pip uninstall pydub
pip install --no-cache-dir --force-reinstall git+https://github.com/jiaaro/pydub.git
이 버전은 최신 GitHub 저장소에서 audioop 없이도 동작하도록 패치된 버전이야!

#### 하지만 결국 이 방법도 작동하지는 않았음.